In [ ]:
import pandas as pd
import re
import nltk
import numpy as np


import pandas as pd
import numpy as np
import nltk
import nltk.corpus
import nltk.tokenize
import string
import gensim
from string import punctuation
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

Текст

In [ ]:
df = pd.read_csv('sent.csv',header=None, sep=',', names=['audio_id','name','start','end', 'pitch',\
                                                                      'text','val'])
df

,audio_id,name,start,end,pitch,text,val
0,1,Юно Гасай,15.1,18.3,220.7,Это твое будущее,0
1,1,Юно Гасай,19.5,24.2,188.8,"Так значит, это правда",.+
2,1,Юно Гасай,40.4,44.5,195.5,От меня не убежишь,-
3,1,Юно Гасай,45.8,49.0,240.0,"Неужели ты еще не понял, Юки?",.+


In [ ]:
df = df[["name", "text"]]
df

,name,text
0,Юно Гасай,Это твое будущее
1,Юно Гасай,"Так значит, это правда"
2,Юно Гасай,От меня не убежишь
3,Юно Гасай,"Неужели ты еще не понял, Юки?"


In [ ]:
def extract_words(text):
  r = re.compile("[а-яА-Я]+")
  words = re.split(r'[,\s]\s*', text)
  words = [re.sub(r'[^\w\s]','', w) for w in words]
  words = [w for w in filter(r.match, words)]
  return words

df['words'] = ""

df['words'] = df.apply(lambda x: extract_words(x['text']), axis=1)

print(df)

        name                           text  \
0  Юно Гасай               Это твое будущее   
1  Юно Гасай         Так значит, это правда   
2  Юно Гасай             От меня не убежишь   
3  Юно Гасай  Неужели ты еще не понял, Юки?   

                                words  
0                [Это, твое, будущее]  
1          [Так, значит, это, правда]  
2             [От, меня, не, убежишь]  
3  [Неужели, ты, еще, не, понял, Юки]  


In [ ]:
!pip install pymorphy2
import pymorphy2
from nltk.corpus import stopwords
nltk.download("stopwords")

stop_words = stopwords.words('russian') # удаление стоп слов

morph = pymorphy2.MorphAnalyzer()

df['norm_words'] = df.apply(lambda x: [morph.parse(word)[0].normal_form for word in x['words'] if morph.parse(word)[0].normal_form not in stop_words], axis=1)
df['tags'] = df.apply(lambda x: [morph.parse(word)[0].tag for word in x['words'] if morph.parse(word)[0].normal_form not in stop_words], axis=1)

print(df)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7d0f030c3730018404fd71af41efb27dd3dfc60dcb861cb6dc2a116e558bc4f4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
        name                           text  \
0  Юно Гасай               Это твое будущее   
1  Юно Гасай         Так значит, это правда   
2  Юно Гасай             От меня не убежишь   
3  Юно Гасай  Неужели ты еще не понял, Юки?   

                                words                   norm_words  \
0                [Это, твое, будущее]         [это, твой, будущее]   
1          [Так, значит, это, правда]        [значит, это, правда]   
2             [От, меня, не, убежишь]                    [убежать] 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Тональность

In [ ]:
df_word_rating = pd.read_csv('words_all_full_rating.csv', encoding='cp1251', sep=';')[["Words", "mean"]]
df_word_rating['mean'] = df_word_rating.apply(lambda x: float(x['mean'].replace(',', '.')), axis=1)
df_word_rating

,Words,mean
0,аборигенный,-0.250000
1,аборт,-1.000000
2,абрамович,0.000000
3,абсолютный,0.333333
4,абстрактный,-0.111111
...,...,...
7540,ярый,-0.333333
7541,ясно,0.000000
7542,ясность,0.666667
7543,ясный,0.666667


In [ ]:
df_coding_result = df[['words','norm_words']]

df_coding_result['coding_words'] = df_coding_result.apply(lambda x: \
  [float(df_word_rating.loc[df_word_rating['Words'] == word]['mean']) for word in x['norm_words'] if df_word_rating['Words'].isin([word]).any()], axis=1)

df_coding_result

<ipython-input-10-905f519812bb>:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  [float(df_word_rating.loc[df_word_rating['Words'] == word]['mean']) for word in x['norm_words'] if df_word_rating['Words'].isin([word]).any()], axis=1)
<ipython-input-10-905f519812bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coding_result['coding_words'] = df_coding_result.apply(lambda x: \


,words,norm_words,coding_words
0,"[Это, твое, будущее]","[это, твой, будущее]",[]
1,"[Так, значит, это, правда]","[значит, это, правда]",[0.333333333333333]
2,"[От, меня, не, убежишь]",[убежать],[]
3,"[Неужели, ты, еще, не, понял, Юки]","[неужели, ещё, понять, юки]",[]


In [ ]:
def set_none(df):
 for i, row in df.iterrows():
    if not row['coding_words']:
      df.loc[i,'coding_words'] = [float(0)]
 return df

#заполнение пустых списков нулем
df_coding_result = set_none(df_coding_result)
df_coding_result

,words,norm_words,coding_words
0,"[Это, твое, будущее]","[это, твой, будущее]",[0.0]
1,"[Так, значит, это, правда]","[значит, это, правда]",[0.333333333333333]
2,"[От, меня, не, убежишь]",[убежать],[0.0]
3,"[Неужели, ты, еще, не, понял, Юки]","[неужели, ещё, понять, юки]",[0.0]


In [ ]:
df_coding_result['coding_mean'] = df_coding_result.apply(lambda x: sum(x['coding_words']) / len(x['coding_words']), axis=1)
df_coding_result['coding_max'] = df_coding_result.apply(lambda x: max(x['coding_words']), axis=1)
df_coding_result['coding_min'] = df_coding_result.apply(lambda x: min(x['coding_words']), axis=1)
df_coding_result['coding_sum'] = df_coding_result.apply(lambda x: sum(x['coding_words']), axis=1)
df_coding_result['coding_count_pos'] = df_coding_result.apply(lambda x: sum(1 for word in x['coding_words'] if word > 0), axis=1)
df_coding_result['coding_count_neg'] = df_coding_result.apply(lambda x: sum(1 for word in x['coding_words'] if word < 0), axis=1)
df_coding_result

<ipython-input-12-8522e07449e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coding_result['coding_mean'] = df_coding_result.apply(lambda x: sum(x['coding_words']) / len(x['coding_words']), axis=1)


,words,norm_words,coding_words,coding_mean,coding_max,coding_min,coding_sum,coding_count_pos,coding_count_neg
0,"[Это, твое, будущее]","[это, твой, будущее]",[0.0],0.000000,0.000000,0.000000,0.000000,0,0
1,"[Так, значит, это, правда]","[значит, это, правда]",[0.333333333333333],0.333333,0.333333,0.333333,0.333333,1,0
2,"[От, меня, не, убежишь]",[убежать],[0.0],0.000000,0.000000,0.000000,0.000000,0,0
3,"[Неужели, ты, еще, не, понял, Юки]","[неужели, ещё, понять, юки]",[0.0],0.000000,0.000000,0.000000,0.000000,0,0
